In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
import numpy as np
import os
import random

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.contrib import layers, rnn, seq2seq
from tqdm import tqdm

In [3]:
import fasttext as fs

In [4]:
from data_generator import NLUDataGenerator
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple
from tensorflow.contrib.legacy_seq2seq import rnn_decoder

In [5]:
PAD = 0

TimeMajor = False
batch_size = 32
encoder_max_time = 64

DataGenDev = NLUDataGenerator('../data/dev/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', None, seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

DataGen = NLUDataGenerator('../data/train/usr_df_final.csv',
                           '../data/ontology_dstc2.json',
                           '../data/slots.txt', DataGenDev.vocab, seq_len = encoder_max_time,
                           batch_size = batch_size, time_major=TimeMajor)

vocab_size = len(DataGen.vocab)
input_embedding_size = 100
# encoder_hidden_units = 128
# decoder_hidden_units = encoder_hidden_units * 2

len(diff):  175
tmp len:  175
before:  711
fin:  886


In [6]:
x, m1, m2, _lens = next(DataGen)
for x_, m1_, m2_ in zip(x, m1, m2):
    print(DataGen.decode_sentence(x_))
    print(DataGen.decode_acts(m2_))
    print(DataGen.decode_slots(m1_))
    print("-----")

['can', 'i', 'have', 'the', 'adddress', 'phone', 'number', 'and', 'area']
['O', 'O', 'O', 'O', 'B-request', 'B-request', 'I-request', 'O', 'B-request']
['O', 'O', 'O', 'O', 'B-addr', 'B-phone', 'I-phone', 'O', 'B-area']
-----
['i', 'want', 'to', 'find', 'a', 'restaurant', 'in', 'the', 'center', 'that', 'should', 'serve', 'romanian', 'food']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-inform', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-food', 'O']
-----
['what', 'was', 'the', 'phone', 'for', 'chiquito']
['O', 'O', 'O', 'B-request', 'O', 'O']
['O', 'O', 'O', 'B-phone', 'O', 'O']
-----
['um', 'what', 'about', 'malaysian', 'food']
['O', 'O', 'O', 'B-inform', 'O']
['O', 'O', 'O', 'B-food', 'O']
-----
['i', 'want', 'to', 'find', 'a', 'restaurant', 'in', 'the', 'center', 'that', 'serves', 'brazilian', 'food']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-inform', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-food', '

In [7]:
slots_number = len(DataGen.slots_encode)
acts_number = len(DataGen.acts_encode)
print("slots_number: ", slots_number)
print("acts_number: ", acts_number)
print("vocab_size: ", DataGen.vocab_size)

slots_number:  26
acts_number:  18
vocab_size:  886


In [8]:
# MAKING EMBEDDING MATRIX

fs_model = fs.load_model("./model.bin")
embedd = np.zeros((vocab_size, input_embedding_size), dtype=np.float32)
for k, v in DataGen.vocab.items():
    embedd[v] = fs_model[k]
embedd[0] = [0]*100

In [25]:
tf.reset_default_graph()

lstm_size = 256
inputs = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask1 = tf.placeholder(shape=(None, None), dtype=tf.int32)
mask2 = tf.placeholder(shape=(None, None), dtype=tf.int32)
lens = tf.placeholder(shape=(None,), dtype=tf.int32)

embeddings = tf.get_variable('embeddings', initializer=embedd, dtype=tf.float32, trainable=False)

encoded = tf.nn.embedding_lookup(embeddings, inputs)
cell = LSTMCell(lstm_size)

outputs, _ = tf.nn.dynamic_rnn(cell, encoded, sequence_length=lens, dtype=tf.float32)


seq_mask = tf.to_float(tf.greater(inputs, PAD))
# with tf.variable_scope('lm-reg'):
#     first_words = tf.slice(inputs, [0, 1], [-1, -1])
#     target = tf.pad(first_words, [[0, 0], [0, 1]])
#     nwp = layers.fully_connected(outputs, vocab_size, activation_fn=None)
#     lm_loss = seq2seq.sequence_loss(nwp, target, seq_mask)

with tf.variable_scope('mask-pred'):
    m1p = layers.fully_connected(outputs, slots_number, activation_fn=None)
    m2p = layers.fully_connected(outputs, acts_number, activation_fn=None)
    
    prediction_m1 = tf.argmax(tf.nn.softmax(m1p), axis = 2)
    prediction_m2 = tf.argmax(tf.nn.softmax(m2p), axis = 2)
    
    m1_loss = tf.reduce_sum(seq2seq.sequence_loss(m1p, mask1, seq_mask, average_across_batch=False, average_across_timesteps=False))
#     m2_loss = tf.reduce_sum(seq2seq.sequence_loss(m2p, mask2, seq_mask, average_across_batch=False, average_across_timesteps=False)

#     log_likelihood_1, transition_params_1 = tf.contrib.crf.crf_log_likelihood(m1p, mask1, lens)
#     log_likelihood_2, transition_params_2 = tf.contrib.crf.crf_log_likelihood(m2p, mask2, lens)
    
lr = tf.Variable(0.0001, trainable=False)
opt = tf.train.GradientDescentOptimizer(lr)
# loss = tf.reduce_mean(-log_likelihood_1)
train_op = opt.minimize(m1_loss)

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    avg_loss = -np.ones((3, ))
    alpha = 0.9
    
    pbar = tqdm(range(100000))
    for step in pbar:
        x, m1, m2, _lens = next(DataGen)
        res = sess.run([m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
        pbar.set_description(str(res[0]))
#         print(res[0])
#         res = sess.run([lm_loss, m1_loss, m2_loss, train_op], {inputs: x, mask1: m1, mask2: m2, lens: _lens})
#         res = np.array(res[:-1])
        
#         if avg_loss[0] < 0:
#             avg_loss = res
        
#         avg_loss = alpha * avg_loss + (1-alpha) * res
        
#         pbar.set_description(' '.join('{:.2f}'.format(_) for _ in avg_loss))
        
        
        if(step%1000 == 0):
            
            x_dev, m1_dev, m2_dev, _lens_dev = next(DataGenDev)
            print("step: ", step)
            
            N = 3 # how many examples show to test:
            loss_cv_1, _, pred_1 = sess.run([m1_loss, train_op, prediction_m1], {inputs: x_dev, 
                                                      mask1: m1_dev,
                                                      lens: _lens_dev})
            print("CV_loss_slots: ", loss_cv_1)
#             print("CV_loss_acts: ", m2_cv_loss)
            
            for x_, m1_, p1 in zip(x_dev[:N], m1_dev[:N], pred_1[:N]):

                print("INPUT: ", DataGen.decode_sentence(x_))
#                 print("m2_tar: ", DataGen.decode_acts(m2_))
#                 print("m2_pred: ", DataGen.decode_acts(p2))                
                
                print("-----")
                
                print("m1_tar: ", DataGen.decode_slots(m1_))
                print("m1_pred: ", DataGen.decode_slots(p1))
                
                print("==========================")
            



  0%|          | 0/100000 [00:00<?, ?it/s]

634.559:   0%|          | 1/100000 [00:00<4:15:37,  6.52it/s]

567.216:   0%|          | 6/100000 [00:00<3:09:33,  8.79it/s]

step:  0
CV_loss_slots:  633.044
INPUT:  ['okay', 'can', 'i', 'have', 'romanian', 'food']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-food', 'O']
m1_pred:  ['B-postcode', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['number', 'number']
-----
m1_tar:  ['B-phone', 'B-phone']
m1_pred:  ['I-area', 'I-area', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['pan', 'asian', 'food']
-----
m1_tar:  ['O', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']




490.6:   0%|          | 11/100000 [00:00<2:24:05, 11.57it/s] 

409.369:   0%|          | 16/100000 [00:00<1:52:34, 14.80it/s]

363.178:   0%|          | 21/100000 [00:00<1:30:04, 18.50it/s]

381.396:   0%|          | 26/100000 [00:00<1:14:08, 22.47it/s]

395.981:   0%|          | 31/100000 [00:00<1:03:24, 26.28it/s]

310.506:   0%|          | 36/100000 [00:00<56:15, 29.61it/s]  

295.597:   0%|          | 41/100000 [00:01<51:05, 32.61it/s]

328.761:   0%|          | 46/100000 [00:01<48:05, 34.64it/s]

358.085:   0%|          | 51/100000 [00:01<45:25, 36.67it/s]

320.175:   0%|          | 56/100000 [00:01<43:04, 38.66it/s]

311.44:   0%|          | 61/100000 [00:01<40:17, 41.35it/s] 

348.069:   0%|          | 66/100000 [00:01<38:36, 43.15it/s]

350.808:   0%|          | 71/100000 [00:01<38:21, 43.42it/s]

376.452:   0%|          | 76/100000 [00:01<38:01, 43.79it/s]

280.693:   0%|          | 81/100000 [00:01<38:56, 42.76it/s]

327.737:   0%|          | 86/100000 [00:02<38:24, 43.35it

214.761:   1%|          | 1007/100000 [00:22<37:17, 44.24it/s]

step:  1000
CV_loss_slots:  214.217
INPUT:  ['i', 'want', 'to', 'find', 'a', 'cheap', 're']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'O', 'B-pricerange', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['im', 'looking', 'for', 'a', 'crossover', 'food', 'restaurant']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-food', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['food', 'in', 'the', 'south', 'part', 'of', 'town']
-----
m1_tar:  ['O', 'O', 'O', 'B-area', 'O', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


...

22.4659:  42%|████▏     | 42006/100000 [13:54<22:36, 42.76it/s]

step:  42000
CV_loss_slots:  36.0007
INPUT:  ['vegitarian']
-----
m1_tar:  ['O']
m1_pred:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['and', 'phone']
-----
m1_tar:  ['O', 'B-phone']
m1_pred:  ['O', 'B-phone', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
INPUT:  ['im', 'looking', 'for', 'an', 'moderate', 'restaurant', 'in', 'the', 'east', 'part', 'of', 't']
-----
m1_tar:  ['O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O']
m1_pred:  ['O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O', 'O', 'O', 'O']



34.8973:  43%|████▎     | 42519/100000 [14:05<20:34, 46.55it/s]

KeyboardInterrupt: 


27.2669:  43%|████▎     | 42519/100000 [14:23<19:27, 49.25it/s]

In [ ]:
file_writer = tf.summary.FileWriter('../logdir/', sess.graph)

## Это чисто черновик, где происходит какая-то фигня с bi-LSTM

In [ ]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

encoder_targets = tf.placeholder(shape=(None), dtype=tf.int32, name='encoder_targets')
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [ ]:
encoder_cell_b = LSTMCell(encoder_hidden_units)
encoder_cell_f = LSTMCell(encoder_hidden_units, reuse=None)

In [ ]:
# encoder_cell = LSTMCell(encoder_hidden_units, reuse=tf.get_variable_scope().reuse)
# encoder_cell_f = LSTMCell(encoder_hidden_units)
((encoder_fw_outputs,
encoder_bw_outputs),
(encoder_fw_final_state,
encoder_bw_final_state)) = (tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell_f,
                                cell_bw=encoder_cell_b,
                                inputs=encoder_inputs_embedded,
                                sequence_length=encoder_inputs_length,
                                dtype=tf.float32, time_major=TimeMajor))
    

In [ ]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h)

In [ ]:
from tensorflow.contrib import layers
next_word_pred = layers.fully_connected(encoder_outputs, vocab_size, ...)



mask1_pred = layers.fully_connected(encoder_outputs, mask1_vocab, ...)
mask2_pred = layers.fully_connected(encoder_outputs, mask2_vocab, ...)


In [ ]:
encoder_final_state_h

In [ ]:
W = tf.Variable(tf.random_uniform([encoder_hidden_units*2, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [ ]:
logits = tf.add(tf.matmul(encoder_final_state_h, W), b)
# encoder_logits = tf.reshape(encoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [ ]:
logits

In [ ]:
encoder_targets

In [ ]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = encoder_targets)